In [7]:
%pip install -q -r "C:\Users\DELL\Desktop\trainwhatasppllm\requirements.txt"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sys
sys.path.append('..')

In [4]:
from minbpe import BasicTokenizer

tokenizer = BasicTokenizer()
tokenizer.load(model_file=r"C:\Users\DELL\Desktop\trainwhatasppllm\output\tokenizer\my_tokenizer.model")

def get_vocab_size(tokenizer: BasicTokenizer) -> int:
    vocab = tokenizer.vocab
    special_tokens = tokenizer.special_tokens

    return len(vocab) + len(special_tokens)

In [5]:
import torch
torch.manual_seed(3647)

In [10]:
from transformers.model import GPTLanguageModel

block_size = 256
n_embd = 12
n_layer = 4
dropout = 0.2
batch_size = 64
vocab_size = get_vocab_size(tokenizer)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = GPTLanguageModel(
    vocab_size=vocab_size,
    block_size=block_size,
    n_embd=n_embd,
    n_layer=n_layer,
    dropout=dropout,
    device=device
).to(device)
model = torch.compile(model)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

ModuleNotFoundError: No module named 'transformers.model'

In [ ]:
with open("../output/combined_text_txt", "r") as f:
    text_sequence = f.read()

encoded_text_sequence = tokenizer.encode(text_sequence)
len(encoded_text_sequence)

In [1]:
data = torch.tensor(encoded_text_sequence, dtype=torch.long)
split_index = int(0.9*len(data))
train_data = data[:split_index]
val_data = data[split_index:]

NameError: name 'torch' is not defined

In [ ]:
from typing import Tuple
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, data:torch.tensor, block_size:int) -> None:
        self.data = data
        self.block_size = block_size

    def __len__(self) -> int:
        return len(self.data) - self.block_size
    
    def __getitem__(self, index:int) -> Tuple[torch.Tensor, torch.Tensor]:
        x = self.data[index:index + self.block_size]
        y = self.data[index + 1:index + self.block_size + 1]
        return x, y

def get_dataloaders(
        train_data: torch.Tensor,
        val_data: torch.Tensor,
        block_size: int,
        batch_size: int,
        device: torch.device
) -> Tuple[DataLoader, DataLoader]:
    
    train_dataset = TextDataset(train_data.to(device), block_size)
    val_dataset = TextDataset(val_data.to(device), block_size)

    train_loader = DataLoader(
        train_dataset,
        batch_size = batch_size,
        shuffle=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    return train_loader, val_loader

In [ ]:
train_loader, val_loader = get_dataloaders(
    train_data=train_data,
    val_data=val_data,
    block_size=block_size,
    batch_size=batch_size,
    device=device
)

x,y = next(iter(train_loader))
x.shape, y.shape

In [2]:
from typing import Dict

@torch.no_grad()
def estimat_loss(
    model: torch.nn.Module,
    train_laoder: DataLoader,
    val_loader: DataLoader,
    eval_iters: int
) -> Dict[str, float]:
    
    output = {}
    model.eval()

    for split, loader in [('train', train_laoder), ('val', val_loader)]:
        losses = torch.zeros(eval_iters)
        for i, (x,y) in enumerate(loader):
            if i >= eval_iters:
                break
            with torch.no_grad():
                _, loss = model(x,y)
            losses[i] = loss.item()
        output[split] = losses.mean().item()
    
    model.train()
    return output

NameError: name 'torch' is not defined

In [4]:
def save_checkpoint(
        model: GPTLanguageModel,
        optimizer: torch.optim.Optimizer,
        epoch: int,
        loss: float,
        file_path: str = "checkpoint.pth"
) -> None:
    
    checkpoint={
        'epoch': epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        'loss':loss
        }
    
    torch.save(checkpoint, file_path)

NameError: name 'GPTLanguageModel' is not defined

In [ ]:

import torch.optim.adam
import torch.optim.adamw


max_iters = 1
eval_intervals = 100
eval_iters = 200
learning_rate = 3e-4


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
train_loader, val_loader = get_dataloaders(
    train_data=train_data,
    val_data=val_data,
    block_size=block_size,
    batch_size=batch_size,
    device=device
)

train_losses = []
val_losses = []

for iteration in range(max_iters):
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        if batch_idx % eval_intervals == 0 or batch_idx == len(train_loader) -1:
            losses = estimat_loss(
                model=model,
                train_laoder=train_loader,
                val_loader=val_loader,
                eval_iters=min(eval_iters, len(val_loader))
            )
            train_losses.append(losses['train'])
            val_losses.append(losses['val'])

            print(
                f"iteration {iteration} / step {batch_idx}: "
                f"train loss {losses['train']:.4f}, "
                f"val loss {losses['val']:.4f}"
            )

        logits, loss = model(x_batch, y_batch)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        loss.step()

    save_checkpoint(model=model,
                    optimizer=optimizer,
                    epoch=iteration,
        loss=loss.item(),
        file_path=f"../output/pre_training/run_4/checkpoint_{iteration}.pth")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3013761366.py, line 47)

In [6]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss", marker='o')
plt.plot(val_losses, label="Validation Loss", marker='o')
plt.xlabel("Evaluation Step")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Time")
plt.legend()
plt.grid()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [7]:
input_tokens = tokenizer.encode("Hi")
input_tokens = torch.tensor(
    input_tokens, dtype=torch.long).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model.generate(input_tokens=input_tokens, max_new_tokens=100)

print(tokenizer.decode(output[0].tolist()))

NameError: name 'tokenizer' is not defined